In [1]:
from PIL import Image, ImageOps
import pytesseract

In [2]:
import os
import cv2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import string

In [37]:
import re

In [4]:
indir = './txt_boxes/test/'
for root, dirs, filenames in os.walk(indir):
    for filename in filenames:
        print('###################' + filename + '###################')
        im = Image.open(indir + filename)
        #region = crop_box(im)
        #text = pytesseract.image_to_string(region, lang='eng', config='--oem 1 --psm 1') #oem 1 means Neural Nets LSTM engine
        text = pytesseract.image_to_string(im, lang='eng')
        print(text)

###################frame46.jpg###################
KISS Mol Roe
W77? 02' 23.1"
###################frame47.jpg###################
KIS eto Roe
W77? 02' 23.1"
###################frame45.jpg###################
KIS Mtoe
W77? 02' 23.3"
###################frame44.jpg###################
LIS aa oX oy at
W77° 02' 24.2"
###################frame50.jpg###################
KISS etl Roe
W77° 02' 21.9"
###################frame40.jpg###################
IRIS aa X aos
77° 02' 30.9"
###################frame41.jpg###################
RIS ea eto Ro
W77? 02' 29.1"
###################frame43.jpg###################
N38° 53' 57.5"
W77° 02' 25.8"
###################frame42.jpg###################
DIS aatoX aol tb
W77? 02' 27.3"
###################frame25.jpg###################
N38° 54' 05.6"
77° 02' 49.9"
###################frame31.jpg###################
KISS Uo
W77° 02' 46.1"
###################frame30.jpg###################
DIS aak oe da Oo
W77° 02' 47.8"
###################frame24.jpg###################
KIS aaher

In [53]:
indir = './txt_boxes/test/'
for root, dirs, filenames in os.walk(indir):
    for filename in filenames:
        print('###################' + filename + '###################')
        image = cv2.imread(indir + filename)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        result = cv2.threshold(gray_image,120,255,cv2.THRESH_BINARY)[1]
        
        #new_image = cv2.threshold(gray_image,190,255,cv2.THRESH_BINARY)[1]
        #kernel = np.ones((1,1), np.uint8)
        #result = cv2.dilate(new_image, kernel, iterations=1)
        
        result = cv2.bitwise_not(result)
        
        #im = Image.open(indir + filename)
        #region = crop_box(im)
        #text = pytesseract.image_to_string(region, lang='eng', config='--oem 1 --psm 1') #oem 1 means Neural Nets LSTM engine
        
        #plt.imshow(result, cmap='Greys')
        #plt.show()
        
        text = pytesseract.image_to_string(result, lang='eng')
        print(text.split('\n')[0])

###################frame46.jpg###################
N38° 53' 56.5"
###################frame47.jpg###################
N38° 53' 56.5"
###################frame45.jpg###################
N38° 53' 56.8"
###################frame44.jpg###################
N38° §3' 57.1"
###################frame50.jpg###################
N38° 53' 55.5"
###################frame40.jpg###################
N38° 53' 59.1"
###################frame41.jpg###################
N38° §3' 58.6"
###################frame43.jpg###################
N38° 53' 57.5"
###################frame42.jpg###################
N38° 53' 58.1"
###################frame25.jpg###################
N38° 54' 05.6"
###################frame31.jpg###################
N38° 54' 04.6"
###################frame30.jpg###################
N38° 54' 05.1"
###################frame24.jpg###################
N38° 54' 06.0"
###################frame32.jpg###################
N38° 54' 03.9"
###################frame26.jpg###################
N38° 54' 05.4"
###################frame2

In [5]:
degree_sign = u'\N{DEGREE SIGN}'
print(degree_sign)

°


In [93]:
indir = './txt_boxes/gps_box/'

image_dict = {}
bad_lats = {}
bad_lons = {}
imgs_w_ie = []
degree_sign = u'\N{DEGREE SIGN}'
remove = string.punctuation + string.ascii_uppercase + degree_sign + '’' + '”'
remove = remove.replace(".", "") # don't remove periods

def lat_txt_to_float(line, bad_lats, dict_key):
    mod_string = line.translate({ord(char): None for char in remove})
    #print(mod_string)
    pattern = re.compile("(\d{2}\s\d{2}\s\d{2}\.\d)")
    if not(re.match(pattern, mod_string)):
        bad_lats[dict_key] = mod_string
    else:
        try:
            return float(mod_string.split(' ')[0]) + float(mod_string.split(' ')[1])/60 + \
                     float(mod_string.split(' ')[2][:4])/3600
        except ValueError as e:
            raise ValueError(f"Check image: {filename.split('/')[-1]}") from e
            
def lon_txt_to_float(line, bad_lons, dict_key):
    mod_string = line.translate({ord(char): None for char in remove})
    #print(mod_string)
    pattern = re.compile("(\d{2,3}\s\d{2}\s\d{2}\.\d)")
    if not(re.match(pattern, mod_string)):
        bad_lons[dict_key] = mod_string
    else:
        try:
            return float(mod_string.split(' ')[0]) + float(mod_string.split(' ')[1])/60 + \
                     float(mod_string.split(' ')[2][:4])/3600
        except ValueError as e:
            raise ValueError(f"Check image: {filename.split('/')[-1]}") from e

loop_counter = 0

for entry in os.scandir(indir):
    loop_counter += 1
    if loop_counter % 100 == 0:
        print("Loop counter: ", loop_counter)
    GPS_dict = {}
    filename = entry.path
    #print('###################' + filename.split('/')[-1] + '###################')
    image = cv2.imread(filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    result = cv2.threshold(gray_image,120,255,cv2.THRESH_BINARY)[1]

    #new_image = cv2.threshold(gray_image,190,255,cv2.THRESH_BINARY)[1]
    #kernel = np.ones((1,1), np.uint8)
    #result = cv2.dilate(new_image, kernel, iterations=1)

    result = cv2.bitwise_not(result)

    #im = Image.open(indir + filename)
    #region = crop_box(im)
    #text = pytesseract.image_to_string(region, lang='eng', config='--oem 1 --psm 1') #oem 1 means Neural Nets LSTM engine

    #plt.imshow(result, cmap='Greys')
    #plt.show()
                             
    #text = pytesseract.image_to_string(result, lang='eng', config='-c tessedit_char_whitelist=’WN0123456789.° --oem 0')
    text = pytesseract.image_to_string(result, lang='eng', config="""-c tessedit_char_whitelist=’WN0123456789.°\\'\\" --oem 0""")
    #text = pytesseract.image_to_string(result, lang='eng')
    
    dict_key = filename.split('/')[-1].split('.')[0]
    
    try:
        GPS_dict['Lat'] = lat_txt_to_float(text.split('\n')[0], bad_lats, dict_key)
    except IndexError:
        imgs_w_ie.append({dict_key:('Lat',text)})

    try:
        GPS_dict['Lon'] = lon_txt_to_float(text.split('\n')[1], bad_lons, dict_key)
    except IndexError:
        imgs_w_ie.append({dict_key:('Lon',text)})
    
    image_dict[dict_key] = GPS_dict
    
    #print(text.split('\n')[0])
    
#print(bad_lats)
#print(bad_lons)

Loop counter:  100
Loop counter:  200
Loop counter:  300
Loop counter:  400
Loop counter:  500
Loop counter:  600
Loop counter:  700
Loop counter:  800
Loop counter:  900
Loop counter:  1000
Loop counter:  1100
Loop counter:  1200
Loop counter:  1300
Loop counter:  1400
Loop counter:  1500
Loop counter:  1600
Loop counter:  1700
Loop counter:  1800
Loop counter:  1900
Loop counter:  2000
Loop counter:  2100
Loop counter:  2200
Loop counter:  2300
Loop counter:  2400
Loop counter:  2500
Loop counter:  2600
Loop counter:  2700
Loop counter:  2800
Loop counter:  2900
Loop counter:  3000
Loop counter:  3100
Loop counter:  3200
Loop counter:  3300
Loop counter:  3400
Loop counter:  3500
Loop counter:  3600
Loop counter:  3700
Loop counter:  3800
Loop counter:  3900
Loop counter:  4000
Loop counter:  4100
Loop counter:  4200
Loop counter:  4300
Loop counter:  4400
Loop counter:  4500
Loop counter:  4600
Loop counter:  4700
Loop counter:  4800
Loop counter:  4900
Loop counter:  5000
Loop coun

In [100]:
len(image_dict)

39524

In [94]:
len(bad_lons)

5089

In [139]:
#print a 10-sample of key-value pairs from bad_lons
{k: v for i, (k,v) in enumerate(dict((k, bad_lons[k]) for k in bad_lons.keys()).items()) if i < 10}

{'frame28907': '111 42 21 .6',
 'frame6817': '84 00 35. 2 3',
 'frame14998': '94 35 51 .9',
 'frame17249': '97 52 41 .3',
 'frame19516': '1011128.8.',
 'frame16625': '96 59 241',
 'frame6195': '831052.5 .',
 'frame39188': '122 09 01 .8',
 'frame18146': '99 18 41 .6',
 'frame13013': '91 41 58 4'}

In [140]:
#print a 10-sample of key-value pairs from bad_lats
{k: v for i, (k,v) in enumerate(dict((k, bad_lats[k]) for k in bad_lats.keys()).items()) if i < 10}

{'frame30190': '41  50 43.0',
 'frame8184': '3815 00.1 5',
 'frame6817': '38 04 03. 7  .',
 'frame10532': '38 19 01 .0',
 'frame25589': '4010 21.1',
 'frame13985': '38 57 31 .0',
 'frame30184': '41  50 18.6',
 'frame3953': '37 43 51 .8',
 'frame13013': '38 54 41 .5',
 'frame24867': '39 4113.5 '}

In [95]:
len(bad_lats)

2075

In [101]:
#how many frames have both poorly OCR'd latitude and longitude values
len(bad_lats.keys() & bad_lons.keys())

486

In [103]:
len(image_dict.keys() & bad_lats.keys())

2075

In [104]:
len(image_dict.keys() & bad_lons.keys())

5089

In [121]:
#size of dictionary containing key-value pairs from image_dict for keys that are also found in bad_lats dictionary
len(dict((k, image_dict[k]) for k in (image_dict.keys() & bad_lats.keys())))

2075

In [117]:
#print a 10-sample of key-value pairs from image_dict for keys that are also found in bad_lats dictionary
{k: v for i, (k,v) in enumerate(dict((k, image_dict[k]) for k in (image_dict.keys() & bad_lats.keys())).items()) if i < 10}

{'frame8359': {'Lat': None, 'Lon': 85.89155555555556},
 'frame6845': {'Lat': None, 'Lon': 84.04952777777778},
 'frame7862': {'Lat': None, 'Lon': 85.25738888888888},
 'frame32668': {'Lat': None, 'Lon': 115.90894444444444},
 'frame467': {'Lat': None, 'Lon': 77.46622222222223},
 'frame36963': {'Lat': None, 'Lon': None},
 'frame24656': {'Lat': None, 'Lon': 107.78122222222223},
 'frame6392': {'Lat': None, 'Lon': 83.43738888888889},
 'frame9776': {'Lat': None, 'Lon': 87.869},
 'frame5612': {'Lat': None, 'Lon': 82.36247222222222}}

In [136]:
n_none_lats, n_none_lons, n_none_both = (0,0,0)
#subset image_dict to frames that are NOT found in imgs_w_ie list of dictionaries
missing_keys = set().union(*(d.keys() for d in imgs_w_ie))
image_dict_rstr = dict((k, image_dict[k]) for k in (image_dict.keys() - missing_keys))
#count frames in image_dict that have None values in one or both of Lat and Lon
for gps_dict in image_dict_rstr.values():
    if (gps_dict['Lat'] is None) & (gps_dict['Lon'] is None):
        n_none_both += 1
    elif gps_dict['Lat'] is None:
        n_none_lats += 1
    elif gps_dict['Lon'] is None:
        n_none_lons += 1
print("Only Lat none:", n_none_lats, "\nOnly Lon none:", n_none_lons, "\nBoth none:", n_none_both)

Only Lat none: 1571 
Only Lon none: 4603 
Both none: 486


In [137]:
len(image_dict_rstr)

39506

In [138]:
#subset bad_lats dictionary to frames that are NOT found in imgs_w_ie list of dictionaries
missing_keys = set().union(*(d.keys() for d in imgs_w_ie))
bad_lats_rstr = dict((k, image_dict[k]) for k in (bad_lats.keys() - missing_keys))
len(bad_lats_rstr)

2057

In [96]:
lat_n_digits = {}
for value in bad_lats.values():
    n_digits = sum(c.isdigit() for c in value)
    if n_digits not in lat_n_digits:
        lat_n_digits[n_digits] = 1
    else:
        lat_n_digits[n_digits] += 1
print(lat_n_digits)

{7: 1757, 8: 274, 9: 11, 5: 3, 1: 6, 0: 20, 6: 3, 4: 1}


In [87]:
for key, value in bad_lats.items():
    if sum(c.isdigit() for c in value) not in (7,8):
        print(key, ',', value)

frame12124 , 38 381371 1
frame147 , 1
frame37987 , 47 0145.41 5
frame4623 , 38 130211 1
frame8151 , 381419.5 5 1
frame2108 , 38 03 2411 1
frame9470 , 38 10 2570 3
frame8961 , 38 124941 1
frame22008 , 
frame5473 , 38 25 1
frame8619 , 38 143013 1
frame39538 , 
frame39539 , 
frame39103 , 47 331177 5
frame22735 , 1
frame5589 , 38 251911 1
frame3271 , 37 47 15. 
frame652 , 
frame39529 , 
frame39531 , 
frame5541 , 38 24 58 
frame39530 , 
frame12010 , 
frame39532 , 
frame29292 , 7
frame39533 , 
frame9755 , 38 11 33 01 1
frame39537 , 
frame39536 , 
frame8128 , 38114010.0
frame39534 , 
frame39051 , 47 32 173.4 3
frame39535 , 
frame4288 , 1
frame4370 , 38 05 
frame24884 , 39 42 010.0 1 
frame4373 ,  05 58.8
frame39021 , 47 32 419. 5 5
frame4428 , 38 06 51 .0 11
frame3347 , 4
frame39544 , 
frame4362 , 38 051773  1
frame4438 , 3807 28.7 51
frame38138 , 47 034 22. 54 5
frame39540 , 
frame23914 , 39 39 010.0 5
frame39541 , 
frame5484 , 38 25 00.  .
frame39543 , 
frame39542 , 
frame23297 , 39 33 4 1


In [97]:
lon_n_digits = {}
for value in bad_lons.values():
    n_digits = sum(c.isdigit() for c in value)
    if n_digits not in lon_n_digits:
        lon_n_digits[n_digits] = 1
    else:
        lon_n_digits[n_digits] += 1
print(lon_n_digits)

{8: 2769, 7: 1912, 10: 16, 5: 8, 9: 154, 0: 157, 6: 72, 11: 1}


In [98]:
lon_8_digits = {'8, starts with 1': 0, '8, starts with other': 0}
for value in bad_lons.values():
    n_digits = sum(c.isdigit() for c in value)
    if n_digits == 8:
        first_digit_1 = (value[0] == '1')
        if first_digit_1:
            lon_8_digits['8, starts with 1'] += 1
        else:
            lon_8_digits['8, starts with other'] += 1
print(lon_8_digits)

{'8, starts with 1': 2454, '8, starts with other': 315}


In [92]:
for key, value in bad_lons.items():
    if sum(c.isdigit() for c in value) == 9:
        print(key, ',', value)

frame36536 , 1191431.81
frame22543 , 105 25 3515
frame38916 , 121 491832
frame36873 , 119 44 2313
frame22351 , 1051118.61
frame36324 , 1191815.31
frame36325 , 1191810.05
frame7671 , 85 001774 1
frame19661 , 101 25.12.33
frame38877 , 121  46 27.6.1
frame19932 , 101 5213.33
frame20380 , 102 3410.03
frame19663 , 101  25 24.31
frame38861 , 121 44. 47.94
frame39202 , 122 09. 50.41
frame38450 , 12116 23.77
frame38322 , 121 05 010.0.
frame27564 , 110 23. 54.41
frame25400 , 108 12 010.0
frame24087 , 107 11.21.87
frame37560 , 120 28 3251
frame39217 , 122 100311
frame19662 , 101 2918.31
frame28246 , 11115 24.65
frame19058 , 100 41 08 31
frame38531 , 121 21  13.01
frame24124 , 107 13. 25.71
frame19276 , 100 54 010.0
frame20420 , 102 33. 40.21
frame28256 , 11115 39.21
frame19666 , 101 25. 42.33
frame36336 , 1191710.21
frame39010 , 121 57. 03.51
frame22418 , 105 1558.41
frame35414 , 118 27. 52.03
frame27819 , 110 45. 02.71
frame18828 , 100 1335.01
frame28080 , 111 0917.82
frame31512 , 114 1628.31
f

In [99]:
len(imgs_w_ie)

18

In [76]:
imgs_w_ie

[{'frame22008': ('Lon', '')},
 {'frame39538': ('Lon', '')},
 {'frame39539': ('Lon', '')},
 {'frame39529': ('Lon', '')},
 {'frame39531': ('Lon', '')},
 {'frame39530': ('Lon', '')},
 {'frame12010': ('Lon', '')},
 {'frame39532': ('Lon', '')},
 {'frame39533': ('Lon', '')},
 {'frame39537': ('Lon', '')},
 {'frame39536': ('Lon', '')},
 {'frame39534': ('Lon', '')},
 {'frame39535': ('Lon', '')},
 {'frame39544': ('Lon', '')},
 {'frame39540': ('Lon', '')},
 {'frame39541': ('Lon', '')},
 {'frame39543': ('Lon', '')},
 {'frame39542': ('Lon', '')}]

In [127]:
#extract keys of all dictionaries present in imgs_w_ie list of dictionaries
all_keys = set().union(*(d.keys() for d in imgs_w_ie))

In [128]:
all_keys

{'frame12010',
 'frame22008',
 'frame39529',
 'frame39530',
 'frame39531',
 'frame39532',
 'frame39533',
 'frame39534',
 'frame39535',
 'frame39536',
 'frame39537',
 'frame39538',
 'frame39539',
 'frame39540',
 'frame39541',
 'frame39542',
 'frame39543',
 'frame39544'}

In [38]:
pattern = re.compile("(\d{2,3}\s\d{2}\s\d{2}\.\d)")

In [46]:
if re.match(pattern, '77 14 08.2'):
    print("yes")
else:
    print("no")

yes


In [51]:
if not(re.match(pattern, '77 14 08 2')):
    print("no")

no


In [48]:
#dpi = im.info

#indir = './frames_lmtd/'
#im_full = Image.open(indir + 'frame40.jpg')

#dpi = im_full.info['dpi']

In [106]:
n_stamps_per_frame = {}
for key, value in image_dict.items():
    if len(value) not in n_stamps_per_frame:
        n_stamps_per_frame[len(value)] = 1
    else:
        n_stamps_per_frame[len(value)] += 1
n_stamps_per_frame

{2: 39506, 1: 18}